In [299]:
import sys
import os
import json
import pandas as pd
import numpy as np
import random
import copy
from datetime import datetime
from collections import Counter, defaultdict

# Visualization packages
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch

# Append system path
sys.path = [p for p in sys.path if not p.endswith('../..')]  # Cleans duplicated '../..'
sys.path.insert(0, '../')  # This adds `src` to the path

from helpers import io, filters, constants
from analysis import analysis_util, analysis_constants, visualization_util
from web_analysis import parse_robots
from web_analysis import robots_util, forecasting_util


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Define Paths to all relevant files

In [300]:
FPATH_TO_RELEVANT_URL_TOKENS = 'pretrain_data/relevant_url_token_counts.csv'
FPATH_to_HEAD_ROBOTS = "robots_data/temporal_robots_head.json"
FPATH_TO_RAND_ROBOTS = "robots_data/temporal_robots_rand_10k.json"
FPATH_TO_TOS_DATA = "robots_data/tos_ai_scraping_policies.json"
DIRPATHS_TO_ANNOTATED_TASKS = ["annotated_websites/Task 1", "annotated_websites/Task 2"]
START_DATES = "robots_data/domain_start_dates.json"

ALL_COMPANIES_TO_TRACK = ["Google", "OpenAI", "Anthropic", "Cohere", "Common Crawl", "Meta", "Internet Archive", "Google Search", "False Anthropic"]
COMPANIES_TO_ANALYZE = ["Google", "OpenAI", "Anthropic", "Cohere", "Common Crawl", "Meta"]
TEMPORAL_ANALYSIS_START_DATE = '2016-01-01'
TEMPORAL_ANALYSIS_END_DATE = '2024-04-30'

### Load all URL splits (top vs random) and maps to Token Counts

In [301]:
url_token_lookup = robots_util.URLTokenLookup(FPATH_TO_RELEVANT_URL_TOKENS) # 'c4', 'rf', 'dolma'
c4_url_to_counts = url_token_lookup.get_url_to_token_map("c4")
rf_url_to_counts = url_token_lookup.get_url_to_token_map("rf")
dolma_url_to_counts = url_token_lookup.get_url_to_token_map("dolma")
top_c4_urls = url_token_lookup.top_k_urls("c4", 2000)
top_rf_urls = url_token_lookup.top_k_urls("rf", 2000)
top_dolma_urls = url_token_lookup.top_k_urls("dolma", 2000)
random_10k_urls = url_token_lookup.get_10k_random_sample()
all_urls = set(random_10k_urls + top_c4_urls + top_rf_urls + top_dolma_urls)

# Load website snapshots for relevant URLs
website_start_dates = robots_util.read_start_dates(START_DATES, all_urls) # THIS WON'T WORK FOR THE 10k SAMPLE

Number of tokens in 2000 URLs: 18447797380 | 10.85% of c4
Number of tokens in 2000 URLs: 67098747294 | 15.56% of rf
Number of tokens in 2000 URLs: 429152555144 | 21.74% of dolma


### Define Agents and Agent Groups

In [315]:
agent_groups_to_track = robots_util.get_bot_groups(ALL_COMPANIES_TO_TRACK)
agent_subset_to_track = robots_util.get_bot_groups(COMPANIES_TO_ANALYZE)
agents_to_track = robots_util.get_bots()

### Load Robots.txt info

In [303]:
# URL -> Date -> Robots.txt raw text
head_robots = io.read_json(FPATH_to_HEAD_ROBOTS)
random_10k_robots = io.read_json(FPATH_TO_RAND_ROBOTS)
joined_robots = copy.deepcopy(head_robots)
joined_robots.update(random_10k_robots)
robots_util.print_out_robots_info(head_robots)
robots_util.print_out_robots_info(random_10k_robots)

# {URL --> Date --> Agent --> Status}
url_robots_summary, agent_counter_df = robots_util.compute_url_date_agent_status(
    data=joined_robots, 
    # relevant_agents=agents_to_track)
    relevant_agents=[v for vs in agent_groups_to_track.values() for v in vs])

agent_counter_df.to_csv("all_agents_counter.csv", index=False)

Num robot URLs loaded: 830
Earliest time: 2016-01-01
Last time: 2024-04-19
Num robot URLs loaded: 6230
Earliest time: 2016-01-01
Last time: 2024-04-19


In [304]:
url_robots_summary_detailed = robots_util.compute_url_date_agent_status_detailed(
    data=joined_robots, 
    relevant_agents=[v for vs in agent_groups_to_track.values() for v in vs]
)

### Load ToS info

In [305]:
# URL --> Date --> ToS-suburl --> {"verdict": X, "evidence": Y}
tos_policies = io.read_json(FPATH_TO_TOS_DATA)
print(f"Num ToS URLs: {len(tos_policies)}")

Num ToS URLs: 3068


### Load Manual Pretraining Annotations

In [306]:
url_to_info = analysis_util.extract_url_annotations(DIRPATHS_TO_ANNOTATED_TASKS)
url_results_df = analysis_util.process_url_annotations(url_to_info)
url_results_df = analysis_util.encode_size_columns(url_results_df, url_token_lookup)
url_results_df = robots_util.encode_latest_tos_robots_into_df(
    url_results_df, tos_policies, url_robots_summary,
    COMPANIES_TO_ANALYZE
)

6663 rows before filtering.
3017 rows after filtering. 1550 issues, 2096 unannotated.
<class 'collections.defaultdict'>
7059


# Create Plots

### Preprocessing for Robots Head & Random URL splits

In [307]:
### DECISION POINT: Use C4, Dolma, or RefinedWeb here?

CHOSEN_CORPUS = "c4" # 'c4', 'rf', 'dolma'
if CHOSEN_CORPUS == "c4":
    HEAD_URL_SET = top_c4_urls
    URL_TO_COUNTS = c4_url_to_counts
elif CHOSEN_CORPUS == "rf":
    HEAD_URL_SET = top_rf_urls
    URL_TO_COUNTS = rf_url_to_counts
elif CHOSEN_CORPUS == "dolma":
    HEAD_URL_SET = top_dolma_urls
    URL_TO_COUNTS = dolma_url_to_counts

In [308]:
url_robots_summary_head = {url: url_robots_summary[url] for url in HEAD_URL_SET if url in url_robots_summary}
url_robots_summary_head_detailed = {url: url_robots_summary_detailed[url] for url in HEAD_URL_SET if url in url_robots_summary_detailed}
url_robots_summary_rand = {url: url_robots_summary[url] for url in random_10k_urls if url in url_robots_summary}

In [309]:
# HEAD URL SPLIT
# {Period --> Agent --> Status --> set(URLs)}
robots_filled_status_head_summary = robots_util.prepare_robots_temporal_summary(
    url_robots_summary=url_robots_summary_head, 
    # group_to_agents={k: [k] for k in agents_to_track},
    group_to_agents=agent_groups_to_track,
    start_time=TEMPORAL_ANALYSIS_START_DATE, 
    end_time=TEMPORAL_ANALYSIS_END_DATE,
    time_frequency="M",
    website_start_dates=website_start_dates,
)
# RANDOM URL SPLIT
robots_filled_status_rand_summary = robots_util.prepare_robots_temporal_summary(
    url_robots_summary=url_robots_summary_rand, 
    # group_to_agents={k: [k] for k in agents_to_track},
    group_to_agents=agent_groups_to_track,
    start_time=TEMPORAL_ANALYSIS_START_DATE, 
    end_time=TEMPORAL_ANALYSIS_END_DATE,
    time_frequency="M",
    website_start_dates=website_start_dates,
)

# DETAILED HEAD
robots_filled_status_head_summary_detailed = robots_util.prepare_robots_temporal_summary_detailed(
    url_robots_summary=url_robots_summary_head_detailed, 
    group_to_agents=agent_groups_to_track,
    start_time=TEMPORAL_ANALYSIS_START_DATE, 
    end_time=TEMPORAL_ANALYSIS_END_DATE,
    time_frequency="M",
    website_start_dates=website_start_dates,
)

In [330]:
# Dataframe w/ [Period, Agent, Status, count(URLs or tokens)]
robots_temporal_head_summary = robots_util.robots_temporal_to_df(
    robots_filled_status_head_summary,
    strictness_order=['no_robots', 'none', 'some', 'all'],
    url_to_counts=c4_url_to_counts,
)
# Dataframe w/ [Period, Agent, Status, count(URLs), count(tokens)]
robots_temporal_rand_summary = robots_util.robots_temporal_to_df(
    robots_filled_status_rand_summary,
    strictness_order=['no_robots', 'none', 'some', 'all'],
    url_to_counts=URL_TO_COUNTS,
)

detailed_strictness_order = [
    'no_robots', 'none', 'none_sitemap', 'none_crawl_delay', 'some_pattern_restrictions', 'some_disallow_important_dir', 'some_other', 'all'
]
robots_temporal_head_summary_detailed = robots_util.robots_temporal_to_df(
    robots_filled_status_head_summary_detailed,
    strictness_order=detailed_strictness_order,
    url_to_counts=URL_TO_COUNTS,
)

### Preprocessing for ToS

In [311]:
# URL --> time --> ToS verdict string. 
url_to_time_to_tos_verdict = robots_util.get_tos_url_time_verdicts(tos_policies)
# Period --> Status --> set(URLs)
period_tos_verdict_urls = robots_util.prepare_tos_temporal_summary(
    url_to_time_to_tos_verdict,
    start_time=TEMPORAL_ANALYSIS_START_DATE, 
    end_time=TEMPORAL_ANALYSIS_END_DATE,
    time_frequency="M",
    website_start_dates=website_start_dates,
)
# Dataframe: [Period, Status, Count, Tokens]
tos_summary_df = robots_util.tos_temporal_to_df(
    period_tos_verdict_urls,
    url_set=HEAD_URL_SET,
    url_to_counts=URL_TO_COUNTS,
)

### Run Forecasts

In [337]:
status_colors = {
    "all": "red",
    "some_pattern_restrictions": "#FFA500",
    #"some_disallow_file_types": "#FF8C00",
    "some_disallow_important_dir": "#FF8C00",
    "some_other": "#FFC107",
    "none_crawl_delay": "lightskyblue",
    "none_sitemap": "navy",
    "none": "blue",
    "no_robots": "gray"
}

ordered_statuses = [
    "no_robots", "none", "none_sitemap", "none_crawl_delay", "some_other",
    "some_disallow_important_dir", "some_pattern_restrictions", "all"
]

# Example usage:
df_to_analyze = robots_temporal_head_summary_detailed
forecasting_util.analyze_robots(
    df_to_analyze, 
    "Combined Agent",
    analysis_type='autoregression',
    chosen_corpus=CHOSEN_CORPUS,
    lags=[1, 3, 6, 12, 24, 48],
    status_colors=status_colors,
    ordered_statuses=ordered_statuses,
    val_col="tokens",
    detailed=True,
)

CHOSEN_CORPUS: c4
AGENT: Combined Agent


alt.Chart(...)

### Plot Temporal Robots Area Chart

In [334]:
# for agent in agents_to_track:
#     robots_util.plot_robots_time_map(robots_temporal_summary, agent)

# for group in agent_groups_to_track:
#     print(group)
chart = robots_util.plot_robots_time_map_altair(
    robots_temporal_head_summary, 
    agent_type="Combined Agent", 
    period_col='period', 
    status_col='status', 
    val_col='tokens',  # "count" / "tokens"
    title='Restriction Status over Time', 
    ordered_statuses=['no_robots', 'none', 'some', 'all'], 
    status_colors={'no_robots': 'gray', 'none': 'blue', 'some': 'orange', 'all': 'red'}
)
chart.show()


alt.Chart(...)

In [336]:
ordered_statuses = [
    "no_robots", "none", "none_sitemap", "none_crawl_delay", "some_other",
    "some_disallow_important_dir", "some_pattern_restrictions", "all"
]

status_colors = {
    "all": "red",
    "some_pattern_restrictions": "#FFA500",
    #"some_disallow_file_types": "#FF8C00",
    "some_disallow_important_dir": "#FF8C00",
    "some_other": "#FFC107",
    "none_crawl_delay": "lightskyblue",
    "none_sitemap": "navy",
    "none": "blue",
    "no_robots": "gray"
}

legend_mapping = {
    "all": "Full restrictions",
    "some_pattern_restrictions": "Pattern-based restrictions",
    "some_disallow_file_types": "Disallow specific file types",
    "some_disallow_important_dir": "Disallow private directories",
    "some_other": "Other restrictions",
    "none_crawl_delay": "Crawl delay specified",
    "none_sitemap": "Sitemap provided",
    "none": "No restrictions or sitemap",
    "no_robots": "No Robots.txt"
}

# for group in agent_groups_to_track:
#     print(group)
chart = robots_util.plot_robots_time_map_altair_detailed(
    robots_temporal_head_summary_detailed,
    agent_type="Combined Agent",
    period_col='period',
    status_col='status',
    val_col='tokens',
    title='Restriction Status over Time',
    ordered_statuses=ordered_statuses,
    status_colors=status_colors,
)
chart.show()

alt.Chart(...)

### Plot Temporal ToS Area Chart

In [ ]:
chart = robots_util.plot_temporal_area_map_altair(
    tos_summary_df,
    period_col='period', 
    status_col='status', 
    val_col='tokens',  # "count" / "tokens"
    title='Restriction Status over Time', 
    ordered_statuses=['No Terms Pages', 'No Restrictions', 'Conditional Restrictions', 'Prohibits AI', 'Prohibits Scraping', 'Prohibits Scraping & AI'], 
    status_colors= {
        'No Terms Pages': 'gray', 'No Restrictions': 'blue', 
        'Conditional Restrictions': 'yellow', 'Prohibits AI': 'orange', 
        'Prohibits Scraping': 'red', 'Prohibits Scraping & AI': 'red'
    },
)
chart.show()

In [ ]:

# robots_temporal_head_summary

Restrictions by Company

In [ ]:
# # Dataframe w/ [Period, Agent, Status, count(URLs), count(tokens)]


In [38]:
agent_color_mapping = {
    "Google": "#1f77b4",          # blue
    "OpenAI": "#ff7f0e",          # orange
    "Anthropic": "#2ca02c",       # green
    "Cohere": "#d62728",          # red
    "Common Crawl": "#9467bd",    # purple
    "Meta": "#8c564b",            # brown
    "Internet Archive": "#e377c2",# pink
    "Google Search": "#7f7f7f",   # gray
    "False Anthropic": "#bcbd22"  # yellow
}

visualization_util.plot_company_comparisons_altair(
    robots_temporal_head_summary, color_mapping=agent_color_mapping)


alt.Chart(...)

In [ ]:
# log scale
# x-axis fix

### Create Correlations Tables

In [ ]:
c4_estimates = analysis_util.run_population_analysis(
    url_results_df, 
    top_c4_urls, 
    "c4",
    "pretrain_data/corpus_token_bucket_counts/c4_buckets.csv",
    url_token_lookup,
    verbose=False,
)
rf_estimates = analysis_util.run_population_analysis(
    url_results_df, 
    top_rf_urls, 
    "rf",
    "pretrain_data/corpus_token_bucket_counts/rf_buckets.csv",
    url_token_lookup,
    verbose=False,
)
dolma_estimates = analysis_util.run_population_analysis(
    url_results_df, 
    top_dolma_urls, 
    "dolma",
    "pretrain_data/corpus_token_bucket_counts/dolma_buckets.csv",
    url_token_lookup,
    verbose=False,
)

In [ ]:
url_correlation_df = analysis_util.analyze_url_variable_correlations(
    url_results_df, 
    [100, 500, 2000], 
    c4_estimates,
    rf_estimates,
    dolma_estimates
)

# Convert the dataframe to a LaTeX table
latex_table = url_correlation_df.to_latex(index=True, escape=True, float_format="{:.1f}".format)
print(latex_table)

### Robots & ToS Confusion Matrix

In [ ]:
robots_util.prepare_tos_robots_confusion_matrix(
    tos_policies,
    url_robots_summary,
    COMPANIES_TO_ANALYZE,
    url_token_lookup,
    corpora_choice="dolma",
)

In [ ]:
type(url_robots_summary)

In [325]:
def company_to_company_restrictions(url_robots_summary, companies):
    # Create a dictionary to hold the URL statuses for each company
    url_status_dict = {}
    
    for company in companies:
        # Get the latest URL robot statuses for the company
        agent_names = robots_util.get_bots(company)
        url_status = robots_util.get_latest_url_robot_statuses(url_robots_summary, agent_names)
        url_status_dict[company] = url_status

    # print(url_status_dict["OpenAI"].values())
    # Create a list to hold the conditional probability data
    conditional_prob_data = []
    
    # Compare each pair of companies
    for company_a in companies:
        for company_b in companies:
            if company_a == company_b:
                continue  # Skip self-comparison
            
            status_a = url_status_dict[company_a]
            status_b = url_status_dict[company_b]
            
            total_restricted_a = sum(1 for status in status_a.values() if status == 'all')
            restricted_b_if_a_restricted = sum(1 for url, status in status_a.items() if status == 'all' and status_b.get(url) == 'all')
            
            if total_restricted_a > 0:
                pct_b_restricted_if_a_restricted = round((restricted_b_if_a_restricted / total_restricted_a) * 100, 1)
            else:
                pct_b_restricted_if_a_restricted = 0
            
            conditional_prob_data.append({
                'Company A': company_a,
                'Company B': company_b,
                'pct_a_restricted_if_b_restricted': pct_b_restricted_if_a_restricted
            })
    
    # Convert the conditional probability data to a DataFrame
    df = pd.DataFrame(conditional_prob_data)
    # print(df)
    
    # Plot the confusion matrix
    return visualization_util.plot_confusion_matrix(
        df,
        yaxis_order=companies, 
        xaxis_order=companies,
        text_axis='pct_a_restricted_if_b_restricted',
        color_axis='pct_a_restricted_if_b_restricted',
        yaxis_title="Company A",
        xaxis_title="Company B",
        font_size=20, 
        font_style='sans-serif',
        width=600,
        height=400,
    )

# Assuming robots_util and visualization_util are already imported and configured correctly


In [326]:
company_to_company_restrictions(url_robots_summary_head, ALL_COMPANIES_TO_TRACK)

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


alt.LayerChart(...)

In [ ]:
# url_robots_summary_head['patents.google.com'].keys()

### Plot Num Tokens against Robots Restrictions per Company

In [35]:
size_bucket_to_urls = robots_util.bucket_urls_by_size(
    c4_url_to_counts, 
    bucket_boundaries=[0, 1000, 10000, 50000, 1000000, 10000000, 50000000, 9999999999999999]
)
robots_util.plot_size_against_restrictions(
    url_robots_summary,
    size_bucket_to_urls,
    "OpenAI",
    setting="train"
)

Bucket 0-1000: 4729
Bucket 1000-10000: 4064
Bucket 10000-50000: 1411
Bucket 50000-1000000: 896
Bucket 1000000-10000000: 2441
Bucket 10000000-50000000: 429
Bucket 50000000-9999999999999999: 19
<class 'collections.defaultdict'>
8958
defaultdict(<function plot_size_against_restrictions.<locals>.<lambda> at 0x3b69daef0>, {'0-1000': [113, 1505, 1073], '1000000-10000000': [240, 640, 466], '50000-1000000': [46, 408, 261], '10000-50000': [25, 666, 354], '1000-10000': [48, 1730, 948], '50000000-9999999999999999': [5, 10, 4], '10000000-50000000': [93, 189, 134]})


alt.LayerChart(...)

# Scratch / Notes

In [ ]:
Notes:

* Take All Agents and subtract it from the other plots to see the diffs between agents.
* How do these charts vary with different quantiles for number of tokens. (Behavior diffs for token rich and token poor)

* Incompatability between Robots.txt and ToS? Robots.txt is an encoded ToS for scrapers. Is there more intention detailed in the ToS than the robots?
* Robots and ToS update rate. --> How often that the other is updated within T time (robots and ToS).
* Analysis: Of all websites that restrict at least one AI bot, what other bots do they restrict? E.g. if you restrict cohere, you probaby also restrict OpenAI
P(Cohere restricted | OpenAI restricted)
* Include Midjourney, CCBot, IAbot, etc
* Vertical lines that show when bots get introduced.

* Restrictions are rising across the board.
* Company-wise restrictions
* Compare for each company their scraping restrictions vs RAG restrictions vs AI bot restrictions (OpenAI, Anthropic)

In [ ]:
Data Sources Analysis:

0. Table: Methodology -- what metadata we collected/annotated (and how). [WIP]
1. Figure: Temporal changes in Robots / ToS (somehow over a collection or multiple companies?) [Waiting on ToS]
    C4/RF/(Dolma) vs (2k-Head)/Random vs Company(6)
2. Figure: Robots / ToS contradiction matrix [Waiting on ToS]
    Creative Commons.
3. Table: Robots permission differences by company + Stella(pairwise differences by confusion matrix) [WIP]
4. Table: Robots / ToS / other indicators variability by website num tokens (i.e. head vs tail) [WIP]
5. Figure: Commercial/market copyright concerns and comparison to WildChat + EU AI Act [WIP, waiting on WildChat]

In [ ]:
Datasets Analysis:

(License: NC, C, Unspecified) x (Terms: Unspecified/None, NC, C).
Text =~ 200
Video = 11
Speech ~= 50

0. Tables for Text, Video, Speech [Will: WIP]
1. License & source (terms) restriction differences between Text, Video, Speech. (normalized stacked bar chart) [WIP]
2. Source domains by Text, Video, Speech: scraped, synthetic, crowdsourced, ....  (normalized stacked bar chart) [WIP]
2b. Include pretraining (broken down by modality + user content + illicit content)
3. Creator distribution for Text, Video, Speech by geography and organization type [WIP]

Extension: 


4. Temporal breakdown of license restriction categories by Text, Video, Speech [WIP]
5. License Type by modality for Text, Video, Speech [WIP]
6. Languages / Tasks.
(Will run statistical tests checking if diffs between Text,Video,Speech are significant on all of these. [WIP]

In [ ]:


# People who restrict Anthropic but not OpenAI. <-- public awareness of organizations is the driving force here. 
# Ordered by notoriety. (Confusion matrix)

In [ ]:
nayan = pd.read_csv("test_data/wildchat_annotations.csv")

In [ ]:
# nayan

In [ ]:
def sample_nayan(df, i):
    row = df.iloc[i]
    prompt = row["WildChat Example Prompt"]
    response = row["WildChat Example Response"]
    typs = row["Types of Service"]
    cd = row["Content Domain"]
    print(prompt)
    # print("*************" + response[:100])
    print(cd)
    print()

In [ ]:
# for i in range(30, 60):
#     sample_nayan(nayan, i)